In [ ]:
import os
import random
import time
import json
import csv
import numpy as np
import torch
import glob


# TEST_DATA_DIR
TEST_DATA_DIR = "/src/data/test/samples"

# MODEL_PATH
MODEL_PATH = "/src/weight/best.pt"

OUTPUT_JSON = 'result/jupyter_submission.json'
OUTPUT_TIME_CSV = 'result/jupyter_time_submission.csv'

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(2024)

In [4]:
from ultralytics import YOLO

# --- LOAD MODEL ---
print(f"Loading model from {MODEL_PATH}...")

try:
    model = YOLO(MODEL_PATH)
    model.predict(source=np.zeros((640, 640, 3), dtype=np.uint8), verbose=False, imgsz=640)

    print("Model loaded successfully!")

except Exception as e:
    print(f"Error loading model: {e}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Loading YOLO11 model from /content/drive/MyDrive/ZALO_AI/final_zalo_full/train2/weights/best.pt...
Warming up model...
Model loaded and warmed up successfully!


In [5]:
# Test Case
# TEST_DATA_DIR
subfolders = sorted(glob.glob(os.path.join(TEST_DATA_DIR, '*')))
test_cases = [f for f in subfolders if os.path.isdir(f)]

print(f"Found {len(test_cases)} test cases folders.")
if len(test_cases) > 0:
    print(f"Example case: {test_cases[0]}")

    example_video = os.path.join(test_cases[0], 'drone_video.mp4')
    print(f" - Video path check: {os.path.exists(example_video)}")

Found 6 test cases folders.
Example case: /content/drive/MyDrive/ZALO_AI/public_test/samples/BlackBox_0
 - Video path check: True


In [ ]:
# Inference Loop
all_predicted_time = []
all_results = []

print("Starting inference on Test Cases...")

for case_folder in test_cases:
    case_id = os.path.basename(case_folder)

    video_path = os.path.join(case_folder, 'drone_video.mp4')
    images_dir = os.path.join(case_folder, 'object_images')

    # --- TÍNH GIỜ ---
    t1 = time.time()

    if not os.path.exists(video_path):
        print(f"Warning: {video_path} not found!")
        continue

    # --- PREDICT ---
    results = model.predict(
        source=video_path,
        verbose=False,
        conf=0.25,
        save=False,
        device=0 if torch.cuda.is_available() else 'cpu'
    )

    frame_results = []
    for r in results:
        # r là kết quả của 1 frame
        boxes = r.boxes.xywh.tolist() if r.boxes else []
        cls = r.boxes.cls.tolist() if r.boxes else []

        if boxes:
            frame_results.append({
                "frame_idx": len(frame_results),
                "boxes": boxes,
                "classes": cls
            })


    final_answer = frame_results

    # --- KẾT THÚC TÍNH GIỜ ---
    t2 = time.time()

    predicted_time = int((t2 - t1) * 1000)


    all_predicted_time.append({
        "id": case_id,
        "answer": "predict_done",
        "time": predicted_time
    })

    # Lưu kết quả chi tiết (cho file submission.json)
    all_results.append({
        "id": case_id,
        "prediction": final_answer
    })

    print(f"Processed {case_id} in {predicted_time}ms")

print("Inference completed.")

Starting inference on Test Cases...
WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Processed BlackBox_0 in 153216ms
WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for

In [ ]:
# jupyter_time_submission.csv
# Yêu cầu format 3 cột: id, answer, time
with open(OUTPUT_TIME_CSV, mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['id', 'answer', 'time']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for item in all_predicted_time:
        writer.writerow(item)

print(f"Saved time submission to {OUTPUT_TIME_CSV}")

# jupyter_submission.json
# Nội dung tương tự submission.json
with open(OUTPUT_JSON, 'w', encoding='utf-8') as json_file:
    json.dump(all_results, json_file, ensure_ascii=False, indent=4)

print(f"Saved result submission to {OUTPUT_JSON}")